# Параметрические критерии

In [57]:
import pandas as pd
import scipy
from statsmodels.stats.weightstats import *

В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота. В эксперименте участвовало 50 испытуемых, проходивших собеседование на программу. Каждый из них разговаривал с рекрутером; в конце 34 из 50 бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате.

Во время ожидания 10 из 34 испытуемых экспериментальной группы и 4 из 16 испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно 4.4%. Ведущие заключили, что миф о заразительности зевоты подтверждён.

Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости при альтернативе заразительности зевоты, округлите до четырёх знаков после десятичной точки.

In [30]:
import numpy as np
import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

In [31]:
total = 50
provocation = 34
p_reaction = 10
without_p_reaction = 4

In [99]:
#расчет левой и правой границы доверительного интервала
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)
# расчет зет-статистики
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))
# расчет p-value
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)
# z статистика связанные выборки   
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = list(zip(sample1, sample2))
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = list(zip(sample1, sample2))
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [33]:
# формируем выборку с провокацией
sample1 = np.hstack((np.ones(p_reaction), np.zeros(provocation-p_reaction)))
len(sample1)

34

In [34]:
# формируем выборку без провокации
sample2 = np.hstack((np.ones(without_p_reaction), np.zeros(total-provocation-without_p_reaction)))
len(sample2)

16

In [35]:
# проводим расчет левой и правой границы доверительного интервала
proportions_diff_confint_ind(sample1, sample2)

(-0.2175577216559601, 0.3057930157736072)

In [39]:
# считаем достигаемый уровень значимости при альтернативе заразительности зевоты, округлив 
# до четырёх знаков после десятичной точки
round(proportions_diff_z_test(proportions_diff_z_stat_ind(sample1, sample2), 'greater'), 4)

0.3729

Имеются данные измерений двухсот швейцарских тысячефранковых банкнот, бывших в обращении в первой половине XX века. Сто из банкнот были настоящими, и сто — поддельными.
Загрузите данные:

banknotes.txt

Отделите 50 случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На оставшихся 150 настройте два классификатора поддельности банкнот:

логистическая регрессия по признакам X_1,X_2,X_3
логистическая регрессия по признакам X_4,X_5,X_6

Каждым из классификаторов сделайте предсказания меток классов на тестовой выборке. Одинаковы ли доли ошибочных предсказаний двух классификаторов? Проверьте гипотезу, вычислите достигаемый уровень значимости. Введите номер первой значащей цифры (например, если вы получили 5.5*10^(-8) , нужно ввести 8).

In [41]:
import pandas as pd

In [42]:
data = pd.read_csv('banknotes.txt', sep='\t')
display(data.head(3))

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1


In [18]:
from sklearn import model_selection

In [43]:
# разделяем предикторы и вектор ответов
X_1 = data.drop(['X4', 'X5', 'X6','real'], axis=1)
X_2 = data.drop(['X1', 'X2', 'X3','real'], axis=1)
y = data.loc[:, data.columns == 'real']
display(X_1.head(1))
display(X_2.head(1))
display(y.head(1))

,X1,X2,X3
0,214.8,131.0,131.1


,X4,X5,X6
0,9.0,9.7,141.0


,real
0,1


In [45]:
from sklearn import model_selection

In [48]:
# отделяем в тестовые выборки 50 наблюдений. 
X_1_train, X_1_test, y_1_train, y_1_test = model_selection.train_test_split(X_1, y, test_size=50, random_state=1)
X_2_train, X_2_test, y_2_train, y_2_test = model_selection.train_test_split(X_2, y, test_size=50, random_state=1)

In [51]:
from sklearn.linear_model import LogisticRegression

In [54]:
# создаем объекты класса LogisticRegression и тренеруем их на учебных подвыборках 
LR_123 = LogisticRegression()
LR_456 = LogisticRegression()

LR_123.fit(X_1_train, np.ravel(y_1_train, order='C'))
LR_456.fit(X_2_train, np.ravel(y_2_train, order='C'))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [55]:
# делаем предсказание меток классов на тестовых выборках 
prediction123 = LR_123.predict(X_1_test)
prediction456 = LR_456.predict(X_2_test)

In [60]:
conf_interval_123 = proportion_confint(np.sum(y_1_test), 
                                            len(y_1_test),
                                            method = 'wilson')
conf_interval_456 = proportion_confint(np.sum(y_2_test), 
                                            len(y_2_test),
                                            method = 'wilson')

In [65]:
print round(conf_interval_123[0][0], 4), round(conf_interval_123[1][0], 4)
print round(conf_interval_456[0][0], 4), round(conf_interval_456[1][0], 4)

0.404 0.6703
0.404 0.6703


In [67]:
from sklearn.metrics import accuracy_score

In [74]:
# делаем расчет точности классификации 
print 'accuracy_123', round(accuracy_score(y_1_test, prediction123), 4)
print 'accuracy_456', round(accuracy_score(y_2_test, prediction456), 4)

accuracy_123 0.8
accuracy_456 0.98


In [83]:
# 
error_123 = []
for i in range(len(y_1_test)):
    if y_1_test.values[i] == prediction123[i]:
        error_123.append(1)
    else:
        error_123.append(0)
        
error_456 = []
for i in range(len(y_2_test)):
    if y_2_test.values[i] == prediction456[i]:
        error_456.append(1)
    else:
        error_456.append(0)

In [86]:
print sum(error_123), len(error_123)
print sum(error_456), len(error_456)

40 50
49 50


In [96]:
# достигаемый уровень значимости
print 'p-value'
round(proportions_diff_z_test(proportions_diff_z_stat_rel(error_123, error_456)), 6)

p-value


0.003297

посчитайте 95% доверительный интервал для разности долей ошибок двух классификаторов. Чему равна его ближайшая к нулю граница? Округлите до четырёх знаков после десятичной точки.

In [100]:
proportions_diff_confint_rel(error_123, error_456)

(-0.3000547937203857, -0.059945206279614305)

In [101]:
round(proportions_diff_confint_rel(error_123, error_456)[0], 4)

-0.3001

Ежегодно более 200000 людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет 525 баллов, стандартное отклонение — 100 баллов.

Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — 541.4. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости 0.05 нулевая гипотеза? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.

In [19]:
import numpy as np
from scipy import stats

In [20]:
marks_all = 525.
st_dev = 100.
marks_corses = 541.4
man = 100

In [21]:
def z_test(mean_val, exp_val, st_dev, num):
    standard_error = st_dev / np.sqrt(num)
    return (mean_val - exp_val) / standard_error
def p_val_greater(z_stat):
    return 1 - stats.norm.cdf(z_stat)

In [22]:
z_stat = z_test(marks_corses, marks_all, st_dev, man)
p_val = p_val_greater(z_stat)
round(p_val, 4)

0.0505

Оцените теперь эффективность подготовительных курсов, средний балл 100 выпускников которых равен 541.5. Отвергается ли на уровне значимости 0.05 та же самая нулевая гипотеза против той же самой альтернативы? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.

In [23]:
marks_corses = 541.5
z_stat = z_test(marks_corses, marks_all, st_dev, man)
p_val = p_val_greater(z_stat)
round(p_val, 4)

0.0495